In [3]:
import numpy as np
import pandas as pd
import stageemi
import stageemi.dev.distance_wwmf as distance_wwmf
import xarray as xr
import os

In [8]:
date = '2020012600'
# date = '2020030600'
dir_csv = '../zonageWME/'
list_method_distance = ['compas','agat','compas_asym','agat_asym'] # pour agreger le temps sensible


if date == '2020012600':
    echeance_dict = {
#         '38':[44,12,3,46,43,25,30],
#         '29':[32,39,20,33,13],  
#         '34':[1,5,6,4 ,10, 20,30], 
#         '41':[45,4,44,5,20,30]

        '34':[1]
        
    }
    
if date == '2020030600':
#     echeance_dict = {
#         '38':[29,3,1,4,36],
#         '41':[18],
#         '29':[1,5,3],  
#         '34':[31,6,16,29,30], 
# }
     echeance_dict = {
        '38':[29,4],
        '29':[5], 
         

    }

In [9]:
for dep_id in echeance_dict.keys():
    echeance_list = echeance_dict[dep_id]
    print('dep_id',dep_id)
    ''' lecture du mask '''
    fname_out = '../GeoData/zones_sympo_multiples/'+dep_id+'_mask_zones_sympos.nc'
    if os.path.exists(fname_out): 
        ds_mask = xr.open_dataset(fname_out,chunks={"id":1})
        ds_mask["latitude"]  = ds_mask["latitude"].round(5)
        ds_mask["longitude"] = ds_mask["longitude"].round(5)
    ''' lecture arome '''
    fname = "../WWMF/" + date+'0000__PG0PAROME__'+'WWMF'+'__EURW1S100______GRILLE____0_48_1__SOL____GRIB2.nc'

    ds = xr.open_dataset(fname,chunks={"step":1}).isel(step = echeance_list)
    ds['latitude']  = ds['latitude'].round(5)
    ds['longitude'] = ds['longitude'].round(5)
    
    ds_dep_tot = (ds*ds_mask.mask.sel(id="departement").drop("id"))
    if date == '2020030600':
        ds_dep_tot = ds_dep_tot.rename({'paramId_0':'unknown'})
        
    ds_distance_dict = {}
    for name in list_method_distance:
        ds_distance         = distance_wwmf.get_pixel_distance_dept(ds_dep_tot,name) # rajoute les variables wme_arr et w1_arr
        ds_distance_chunk   = ds_distance.chunk({"step":1}) 
        ds_distance_dict[name] = (ds_distance_chunk * ds_mask.mask).sum(['latitude',"longitude"]).compute()
    print('fin calcul distance')
    
    for icheance,echeance in enumerate(echeance_list):  
        fname_csv_ini = dir_csv+'v2_'+dep_id+'_'+date+'_'+str(echeance)+'.csv'
        if os.path.exists(fname_out):
            fname_csv_final = dir_csv+ 'v3_'+dep_id+'_'+date+'_'+str(echeance)+'.csv'
            df = pd.read_csv(fname_csv_ini)
            for name in list_method_distance:
                distance2change = ds_distance_dict[name].wwmf_2[ds_distance_dict[name].argmin("wwmf_2")].sel(id=df['zone'].to_numpy()).isel(step=icheance).values
                df[name]        = distance2change
            df.to_csv(fname_csv_final, index = False, header=True) 


dep_id 34
fin calcul distance


In [11]:
df

,Unnamed: 0,zone,cible_wme,hss,compas,agat,compas_asym,agat_asym
0,0,3402+3404,18.0,0.279079,8,19,8,19
1,1,3405+3408,14.0,0.211439,6,7,6,21
2,2,3401,4.0,0.496974,3,1,4,7
3,3,3403+3406+3407+3481+3482,2.0,0.588047,3,0,3,0
4,4,3483,NaN,NaN,6,7,6,7


In [4]:
df1 = pd.read_csv(fname_csv_ini)
df1

,Unnamed: 0,zone,cible_wme,hss,compas,agat,compas_asym,agat_asym
0,0,3401,14.0,0.111109,3,7,3,7
1,1,3402,8.0,0.079285,2,0,2,0
2,2,3403,2.0,0.203952,1,0,2,0
3,3,3404+3405+3406+3407+3408+3483+3481+3482,1.0,0.637874,1,0,1,0


In [5]:
df2 = pd.read_csv(fname_csv_final)
df2

,Unnamed: 0,zone,cible_wme,hss,compas,agat,compas_asym,agat_asym
0,0,3401,14.0,0.111109,3,7,3,7
1,1,3402,8.0,0.079285,2,0,2,0
2,2,3403,2.0,0.203952,1,0,2,0
3,3,3404+3405+3406+3407+3408+3483+3481+3482,1.0,0.637874,1,0,1,0
